# Rank prediction using CatBoost Classifier

-- Nishant Ragate

<b><u>Key components of the model</u></b>

Data Preprocessing: After imputing missing values using KNN. We encode categorical variables, and scale numerical features to ensure compatibility with the CatBoost model.

Feature selection: "Name" and "College" are removed and relevant one's are selected based on their importance in predicting the target variable, ensuring that the model focuses on the most informative aspects of the data while reducing complexity and overfitting.

Target feature: The target feature, initially "Round," is later updated to "Pick" to better reflect the desired outcome, which is predicting the order in which players are selected in a draft.

Dataset split: The dataset is split into training and testing sets, with the training set used to train the model and the testing set used to evaluate its performance.


Additionally, a 5-Fold cross-validation strategy is employed during hyperparameter tuning to ensure robustness and generalizability.

In [2]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("data/imputed_data.csv")
print(df.columns)

Index(['Name', 'Position', 'College', 'Round', 'Pick', 'Stat URL', 'Height',
       'Weight', '40 Yard Dash', 'Bench Press', 'Vertical Jump', 'Broad Jump',
       '3 Cone Drill', 'Shuttle', 'conf_abbr', 'games', 'seasons',
       'tackles_solo', 'tackles_assists', 'tackles_total', 'tackles_loss',
       'sacks', 'def_int', 'def_int_yds', 'def_int_td', 'pass_defended',
       'fumbles_rec', 'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced',
       'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'rush_att', 'rush_yds',
       'rush_yds_per_att', 'rush_td', 'scrim_att', 'scrim_yds',
       'scrim_yds_per_att', 'scrim_td', 'Year'],
      dtype='object')


In [2]:
df.head

<bound method NDFrame.head of                  Name Position          College  Round  Pick  \
0       Emmanuel Acho      OLB            Texas      6   204   
1           Joe Adams       WR         Arkansas      4   104   
2        Chas Alecxih       DT       Pittsburgh      0     0   
3     Frank Alexander       DE         Oklahoma      4   103   
4       Antonio Allen        S   South Carolina      7   242   
...               ...      ...              ...    ...   ...   
3679      Luke Wypler        C         Ohio St.      6   190   
3680      Bryce Young       QB          Alabama      1     1   
3681      Byron Young       DT          Alabama      3    70   
3682      Byron Young     EDGE        Tennessee      3    77   
3683    Cameron Young       DT  Mississippi St.      4   123   

                                               Stat URL  Height  Weight  \
0     https://www.sports-reference.com/cfb/players/e...    74.0   238.0   
1     https://www.sports-reference.com/cfb/players/j...    71.0   179.0   
2     https://www.sports-reference.com/cfb/players/c...    76.0   296.0   
3     https://www.sports-reference.com/cfb/players/f...    76.0   270.0   
4     https://www.sports-reference.com/cfb/players/a...    73.0   210.0   
...                                                 ...     ...     ...   
3679  https://www.sports-reference.com/cfb/players/l...    75.0   303.0   
3680  https://www.sports-reference.com/cfb/players/b...    70.0   204.0   
3681  https://www.sports-reference.com/cfb/players/b...    75.0   294.0   
3682  https://www.sports-reference.com/cfb/players/b...    74.0   250.0   
3683  https://www.sports-reference.com/cfb/players/c...    75.0   304.0   

      40 Yard Dash  Bench Press  ...  rec_td  rush_att  rush_yds  \
0             4.64        24.00  ...    5.29    199.20   1282.58   
1             4.51        14.59  ...    8.50      4.00     69.50   
2             5.31        19.00  ...    0.00      1.19      5.20   
3             4.80        24.48  ...    2.17     22.98     75.37   
4             4.58        17.00  ...    1.68    374.69   2061.25   
...            ...          ...  ...     ...       ...       ...   
3679          5.14        25.26  ...    0.14      1.14     -1.96   
3680          4.65        18.86  ...    0.00      1.00      1.02   
3681          4.92        24.00  ...    0.13      1.53      6.99   
3682          4.43        22.00  ...    1.42     67.49    319.31   
3683          5.10        29.53  ...    0.21      1.08     -0.84   

      rush_yds_per_att rush_td  scrim_att  scrim_yds  scrim_yds_per_att  \
0                 8.83   14.91     239.71    1747.91               8.22   
1                11.65    0.00      96.00    1393.50              14.45   
2                -0.68    0.36       1.36       5.55               0.86   
3                 4.12    4.24      36.81     231.59               6.49   
4                 4.94   19.21     420.39    2397.36               6.43   
...                ...     ...        ...        ...                ...   
3679             -2.52    0.14       2.41       5.95              -2.27   
3680              1.00    0.00       1.00       1.02               1.00   
3681              1.28    0.53       2.67      14.08               1.51   
3682              3.01    4.51      95.10     567.86               5.38   
3683             -2.46    0.21       5.61      32.58              -0.22   

      scrim_td  Year  
0        20.20  2012  
1         8.50  2012  
2         0.36  2012  
3         6.41  2012  
4        20.89  2012  
...        ...   ...  
3679      0.28  2023  
3680      0.00  2023  
3681      0.65  2023  
3682      5.94  2023  
3683      0.42  2023  

[3684 rows x 43 columns]>

In [5]:
df.loc[df.Round != 1, "Round"] = 0

# Dropping the columns which donot contribute in prediction
all_X = df.drop(["Name", "Round", "Pick", "College"], axis=1)
all_X = pd.get_dummies(all_X)

# Splitting testing and training sets
train_X = all_X[(all_X.Year != 2023)].drop(["Year"], axis=1)
test_X = all_X[all_X.Year == 2023].drop(["Year"], axis=1)
train_y = df[(df.Year != 2023)].Round
test_y = df[df.Year == 2023].Round

In [ ]:
train_X.head()

In [ ]:
test_X.head()

In [3]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.5 MB/s eta 0:00:00


In [6]:
from catboost import CatBoostClassifier

# Initialize the baseline CatBoost classifier with custom parameters
baseline_catboost = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    l2_leaf_reg=3,
    border_count=32,
    thread_count=4,
    verbose=False
)

# Train the baseline CatBoost model
baseline_catboost.fit(train_X, train_y)

In [7]:
# Make predictions on test data
baseline_preds = baseline_catboost.predict_proba(test_X)

# Ranking done according to the probability scores
count = 1
for i in pd.DataFrame(baseline_preds).sort_values(by=1, ascending=False).index:
    print(f"{count} {df[df.Year == 2023].reset_index().at[i, 'Name']}")
    count += 1

1 Jakorian Bennett
2 Will Anderson Jr.
3 DJ Turner
4 Emmanuel Forbes
5 Anthony Richardson
6 Marvin Mims
7 Bryce Young
8 Isaiah Foskey
9 Carrington Valentine
10 Kelee Ringo
11 Christian Gonzalez
12 Trenton Simpson
13 Darnell Wright
14 Tyler Steen
15 C.J. Stroud
16 Tavius Robinson
17 Deonte Banks
18 YaYa Diaby
19 Rejzohn Wright
20 Tre'Vius Hodges-Tomlinson
21 Josh Downs
22 Lukas Van Ness
23 Cory Trice
24 Darrell Luter Jr.
25 Bijan Robinson
26 Yasir Abdullah
27 Robert Beal
28 Ali Gaye
29 Jalin Hyatt
30 Byron Young
31 Ryan Hayes
32 Quentin Johnston
33 Nathaniel Dell
34 Wanya Morris
35 Jalen Carter
36 Anthony Bradford
37 Adetomiwa Adebawore
38 Malaesala Aumavae-Laulu
39 Matthew Bergeron
40 BJ Ojulari
41 Jonathan Mingo
42 Carter Warren
43 Rakim Jarrett
44 Tyler Scott
45 Broderick Jones
46 John Ojukwu
47 Jon Gaines
48 Gervon Dexter
49 Bryan Bresee
50 Nick Hampton
51 Nolan Smith
52 Felix Anudike-Uzomah
53 Jartavius Martin
54 Terell Smith
55 Jason Taylor II
56 Tanner McKee
57 Anton Harrison
58 

In [10]:
from tabulate import tabulate
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

# Convert predicted probabilities to binary predictions based on a threshold (e.g., 0.5)
predicted_labels = (baseline_preds[:, 1] > 0.5).astype(int)

# Evaluation for ranking metrics
# Sort the predictions based on probability scores
sorted_indices = np.argsort(-baseline_preds[:, 1])
k = 10
num_relevant = sum(test_y)

# Define ranking evaluation functions
def calculate_MRR(sorted_indices, test_y):
    mrr = 0
    for idx, i in enumerate(sorted_indices):
        if test_y.iloc[i] == 1:
            mrr = 1 / (idx + 1)
            break
    return mrr

def calculate_MAP(sorted_indices, test_y):
    ap = 0
    for idx, i in enumerate(sorted_indices):
        if test_y.iloc[i] == 1:
            ap += sum(test_y.iloc[:idx + 1]) / (idx + 1)
    map_score = ap / num_relevant
    return map_score

def calculate_NDCG(sorted_indices, test_y):
    dcg = 0
    idcg = sum(1 / np.log2(np.arange(2, k + 2)))
    for idx, i in enumerate(sorted_indices[:k]):
        if test_y.iloc[i] == 1:
            dcg += 1 / np.log2(idx + 2)
    ndcg = dcg / idcg
    return ndcg

def calculate_PAK(sorted_indices, test_y):
    tp_at_k = sum(test_y.iloc[sorted_indices[:k]])
    precision_at_k = tp_at_k / k
    return precision_at_k

def calculate_RAK(sorted_indices, test_y):
    tp_at_k = sum(test_y.iloc[sorted_indices[:k]])
    recall_at_k = tp_at_k / num_relevant
    return recall_at_k

# Calculate all measurements
baseline_measurements = [
    ("Accuracy", accuracy_score(test_y, predicted_labels)),
    ("ROC AUC Score", roc_auc_score(test_y, baseline_preds[:, 1])),
    ("Mean Reciprocal Rank (MRR)", calculate_MRR(sorted_indices, test_y)),
    ("Mean Average Precision (MAP)", calculate_MAP(sorted_indices, test_y)),
    ("Normalized Discounted Cumulative Gain (NDCG) at k=10", calculate_NDCG(sorted_indices, test_y)),
    ("Precision at k (P@k) at k=10", calculate_PAK(sorted_indices, test_y)),
    ("Recall at k (R@k) at k=10", calculate_RAK(sorted_indices, test_y))
]

# Print measurements in a table format
print("Baseline measurements")
print(tabulate(baseline_measurements, headers=["Metric", "Value"]))

Baseline measurements
Metric                                                   Value
----------------------------------------------------  --------
Accuracy                                              0.897887
ROC AUC Score                                         0.777552
Mean Reciprocal Rank (MRR)                            0.5
Mean Average Precision (MAP)                          0.123912
Normalized Discounted Cumulative Gain (NDCG) at k=10  0.392158
Precision at k (P@k) at k=10                          0.4
Recall at k (R@k) at k=10                             0.137931


In [28]:
from sklearn.model_selection import cross_val_score

# Find the best parameters for CatBoost
best_catboost = CatBoostClassifier(
    iterations=1500,
    learning_rate=0.1,
    depth=10,
    l2_leaf_reg=1,
    border_count=128,
    scale_pos_weight=(len(train_y) - sum(train_y)) / sum(train_y),
    thread_count=4,
    verbose=False
)

# Cross-validation on the best CatBoost model
best_scores = cross_val_score(best_catboost, train_X, train_y, cv=5)

best_catboost.fit(train_X, train_y)

In [29]:
# Make predictions on test data
preds = best_catboost.predict_proba(test_X)

# Ranking done according to the probability scores after tuning the hyperparameters
count = 1
for i in pd.DataFrame(preds).sort_values(by=1, ascending=False).index:
    print(f"{count} {df[df.Year == 2023].reset_index().at[i, 'Name']}")
    count += 1

1 Darnell Wright
2 Anthony Richardson
3 Jakorian Bennett
4 Bryce Young
5 Tyler Steen
6 Richard Gouraige
7 YaYa Diaby
8 C.J. Stroud
9 Byron Young
10 Emmanuel Forbes
11 Nolan Smith
12 Wanya Morris
13 Ryan Hayes
14 Anton Harrison
15 Kelee Ringo
16 Ali Gaye
17 Christian Gonzalez
18 Malaesala Aumavae-Laulu
19 Rejzohn Wright
20 Nathaniel Dell
21 Brandon Hill
22 Marvin Mims
23 Rakim Jarrett
24 Anthony Bradford
25 Matthew Bergeron
26 Carter Warren
27 Sam LaPorta
28 Jay Ward
29 Bijan Robinson
30 Tavius Robinson
31 Jalin Hyatt
32 Isaiah Foskey
33 Jaren Hall
34 DJ Turner
35 Joe Tippmann
36 Zacch Pickens
37 Trenton Simpson
38 Paris Johnson Jr.
39 Rashee Rice
40 Asim Richards
41 Tanner McKee
42 Calijah Kancey
43 Jon Gaines
44 Broderick Jones
45 A.T. Perry
46 Robert Beal
47 Michael Mayer
48 Adetomiwa Adebawore
49 Jalen Carter
50 Tyler Scott
51 Thomas Incoom
52 Darnell Washington
53 Myles Murphy
54 Jerrod Clark
55 Devon Achane
56 Warren McClendon
57 Mazi Smith
58 Carrington Valentine
59 Peter Skorons

In [30]:
# Make predictions on test data
preds = best_catboost.predict(test_X)

# Calculate accuracy
accuracy = (preds == test_y).mean()

# Evaluation metrics
baseline_measurements = [
    ("Accuracy", accuracy)
]

# Print baseline measurements
print("Baseline CatBoost measurements")
print(tabulate(baseline_measurements, headers=["Metric", "Value"]))

Baseline CatBoost measurements
Metric       Value
--------  --------
Accuracy  0.901408


In [31]:
preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [32]:
# Convert predicted probabilities to binary predictions based on a threshold (e.g., 0.5)
predicted_labels = (preds > 0.5).astype(int)

# Evaluation for ranking metrics
# Sort the predictions based on probability scores
sorted_indices = np.argsort(-preds)

# Calculate all measurements
best_rf_measurements = [
	("Accuracy", accuracy_score(test_y, predicted_labels)),
	("ROC AUC Score", roc_auc_score(test_y, baseline_preds[:, 1])),
	("Mean Reciprocal Rank (MRR)", calculate_MRR(sorted_indices, test_y)),
	("Mean Average Precision (MAP)", calculate_MAP(sorted_indices, test_y)),
	("Normalized Discounted Cumulative Gain (NDCG) at k=10", calculate_NDCG(sorted_indices, test_y)),
	("Precision at k (P@k) at k=10", calculate_PAK(sorted_indices, test_y)),
	("Recall at k (R@k) at k=10", calculate_RAK(sorted_indices, test_y))
]
# Print measurements in a table format
print("Best Fit measurements")
print(tabulate(best_rf_measurements, headers=["Metric", "Value"]))



Best Fit measurements
Metric                                                   Value
----------------------------------------------------  --------
Accuracy                                              0.901408
ROC AUC Score                                         0.777552
Mean Reciprocal Rank (MRR)                            0.5
Mean Average Precision (MAP)                          0.103692
Normalized Discounted Cumulative Gain (NDCG) at k=10  0.55778
Precision at k (P@k) at k=10                          0.6
Recall at k (R@k) at k=10                             0.206897


# Comparative Analysis of Baseline and Best-Fit CatBoost Models for Ranking Prediction

The comparison between the baseline and best-fit CatBoost models reveals notable differences in performance across various metrics. In terms of accuracy the best fit model does slightly better than the baseline, and ROC AUC score in both models exhibit similar results. However, significant improvements are observed in the best-fit model for ranking-related metrics. The Mean Reciprocal Rank (MRR) shows a substantial increase, indicating that the best-fit model provides more relevant and accurate predictions at the top of the ranked list compared to the baseline. Similarly, the Mean Average Precision (MAP) and Precision at k (P@k) at k=10 metrics demonstrate considerable enhancements, implying better precision in predicting relevant instances within the top results. Moreover, the Normalized Discounted Cumulative Gain (NDCG) at k=10 reflects a notable improvement, suggesting that the best-fit model produces more relevant results at the top ranks, which is crucial for ranking tasks. Despite these improvements, the recall at k (R@k) at k=10 remains relatively low for both models, indicating a challenge in capturing all relevant instances within the top k results.

Overall, while the baseline model provides reasonable predictive performance, the best-fit Random Forest model significantly enhances the model's ability to accurately rank and prioritize instances, particularly at the top of the list, thereby improving its utility in predicting NFL Draft.
